
## 🧠 What Is Docker Compose?

<span style="color:lightgreen">**Docker Compose**</span> is a **tool for defining and running multi-container Docker applications**.  

It allows you to manage multiple containers — such as:
- A **web application** container (e.g., Flask app)
- A **Redis** container (for caching)
- A **MySQL** or **MongoDB** container (for databases)

These containers can communicate with each other seamlessly using Docker Compose.

---

## 🧩 Why Do We Need Docker Compose?

Let’s say we have a **web application container** that needs:
- A **MySQL** database
- A **Redis** cache
- A **MongoDB** service  

Running all these manually using multiple `docker run` commands would be messy.  
Instead, with Docker Compose, you define everything in a single file:

```bash
docker-compose.yml
````

This file:

* Defines all services (web, Redis, MySQL)
* Builds and runs them together
* Ensures inter-container networking

---

## ⚙️ Step 1: Create the Flask Application

Example: `app.py`

```python
from flask import Flask
from redis import Redis

app = Flask(__name__)
redis = Redis(host='redis', port=6379)

@app.route('/')
def hello():
    count = redis.incr('hits')
    return f"Hello Krish! I've seen you {count} times."
    
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
```

Redis here is used for **in-memory caching**.

---

## ⚙️ Step 2: Create the Dockerfile

This Dockerfile builds the **Flask web container**.

```Dockerfile
FROM python:3.7-alpine

# Set working directory
WORKDIR /code

# Copy all files to the working directory
COPY . /code

# Environment variables for Flask
ENV FLASK_APP=app.py
ENV FLASK_RUN_HOST=0.0.0.0

# Install dependencies
RUN pip install -r requirements.txt

# Expose the port Flask runs on
EXPOSE 5000

# Command to start the app
CMD ["flask", "run"]
```

🟢 <span style="color:lightgreen">**Explanation:**</span>

* `WORKDIR` → Defines the working directory inside the container.
* `ENV` → Sets environment variables.
* `EXPOSE 5000` → Makes port 5000 accessible outside the container.
* `CMD` → Tells Docker how to start the app.

---

## 🧾 Step 3: Create docker-compose.yml

Now, define multiple containers and how they interact.

```yaml
version: '3.0'

services:
  web:
    build: .
    ports:
      - "8000:5000"
    image: web_app
    depends_on:
      - redis
  
  redis:
    image: redis
```

🔸 <span style="color:orange">**Explanation:**</span>

* `web` → Flask web service built from the Dockerfile.
* `redis` → Pulls Redis image directly from Docker Hub.
* `ports` → Maps host port `8000` to container port `5000`.
* `depends_on` → Ensures Redis starts before the web container.

---

## 🏗️ Step 4: Run the Docker Compose Setup

Open your terminal and execute:

```bash
docker-compose up
```

This command:

1. Builds the Flask image
2. Pulls Redis image
3. Creates a **shared network**
4. Runs both containers together

You’ll see logs for both services.

---

## 🧭 Step 5: Check Running Containers

To see all running containers:

```bash
docker ps
```

Output example:

```
CONTAINER ID   IMAGE        PORTS                  NAMES
abcd1234efgh   web_app      0.0.0.0:8000->5000/tcp web_1
ijkl5678mnop   redis        6379/tcp               redis_1
```

✅ Both containers are now up and communicating.

---

## 🌐 Step 6: Test the Application

Visit:

```
http://localhost:8000
```

Output:

```
Hello Krish! I’ve seen you 1 times.
```

Reload a few times, and you’ll see the count increase — <span style="color:lightgreen">Redis is caching the count in memory!</span> ⚡

---

## 🔧 Step 7: Stopping Containers

To stop all services:

```bash
docker-compose stop
```

All containers stop, but images remain intact.

---

## ⚠️ Common Issue Example

If you modify your `app.py` (e.g., change message to “Hello Dockers!”),
you must rebuild the images to see the update:

```bash
docker-compose build
docker-compose up
```

This is because the old container still uses the previous build.

🧱 <span style="color:red">This can be fixed with Docker Volumes!</span>
Volumes help sync your code changes live without rebuilding.
We’ll explore this in the next section.

---

## 🧩 Optional — Adding MySQL or MongoDB

If you want to extend the setup, you can add more services:

```yaml
mysql:
  image: mysql:latest
  environment:
    MYSQL_ROOT_PASSWORD: root
    MYSQL_DATABASE: mydb
    MYSQL_USER: user
    MYSQL_PASSWORD: pass
  ports:
    - "3306:3306"
```

You can now connect MySQL with your web container using the service name `mysql`.

---

## 🧠 Summary

| Step | Description                                         | Command                |
| ---- | --------------------------------------------------- | ---------------------- |
| 1    | Create Flask app (`app.py`)                         | —                      |
| 2    | Write Dockerfile                                    | —                      |
| 3    | Define multi-container setup (`docker-compose.yml`) | —                      |
| 4    | Build and run containers                            | `docker-compose up`    |
| 5    | Check running containers                            | `docker ps`            |
| 6    | Stop containers                                     | `docker-compose stop`  |
| 7    | Rebuild after code change                           | `docker-compose build` |

---

## 🔮 Next Step

In the next session, we’ll learn about <span style="color:orange">**Docker Volumes**</span> —
how to make your container automatically reflect code changes
without rebuilding each time.

---

<span style="color:lightgreen">✨ That’s it! You’ve now mastered how to use Docker Compose for running multi-container apps.</span>

```
```
